In [ ]:
import torch
import numpy as np 
import pandas as pd 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import Dataset, DataLoader, random_split

# ✅ 데이터 준비 
BATCH_SIZE = 1
N_WORKERS = 0 
SEQ_LEN = 30        # 30일치 데이터 
INPUT_SIZE = 4      # 종가, 시작가, 상한가, 하한가 모두 사용 
OUTPUT_SIZE = 4     # 종가, 시작가, 상한가, 하한가 모두 예측
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

train_df = pd.read_csv('Bitcoin Historical Data.csv', encoding = 'utf-8-sig').fillna(0)
val_df = pd.read_csv('Bitcoin Historical Data 240101-250305.csv', encoding = 'utf-8-sig').fillna(0)

print(train_df.head())

class PriceData(Dataset):
    def __init__(self, input_data):
        super(PriceData, self).__init__()
        self.input, self.label, self.rev_norm = self.edit(input_data)

    def __len__(self):
        return len(self.input)
    
    def __getitem__(self, idx):
        input_val = self.input[idx]
        label_val = self.label[idx]
        norm_val  = self.rev_norm[idx]
        return input_val, label_val, norm_val
    
    def edit(self, input_data):
        input_data = input_data.sort_values(by = 'Date', ascending = True)
        input_data = input_data.apply(lambda x : x.str.replace(',','').str.replace('₩',''))
        input_data = input_data[['Price', 'Open', 'High', 'Low']].astype(np.float32)
        tot_input = []; tot_label = []; rev_norm = []
        for idx, row in input_data.iterrows():
            if idx < SEQ_LEN: continue 
        ### input 
            input_val = input_data.iloc[idx-SEQ_LEN:idx, :]
            input_val_norm = (input_val - input_val.min()+ 1e-2) / (input_val.max() - input_val.min()+ 1e-2)
            input_val_norm_torch = torch.tensor(input_val_norm.values, dtype= torch.float32).to(DEVICE)
            tot_input.append(input_val_norm_torch)
        ### label
            label_val = input_data.iloc[idx, :].values
            label_val_norm = (label_val - input_val.min()+ 1e-2) / (input_val.max() - input_val.min()+ 1e-2)
            label_val_norm_torch = torch.tensor(label_val_norm, dtype= torch.float32).to(DEVICE)
            tot_label.append(label_val_norm_torch)
        ### data for re-interpret the normaliaztion 
            rev_norm.append(torch.tensor([input_val.min(), input_val.max()], dtype= torch.float32).to(DEVICE))

        input_data = torch.stack(tot_input)
        label_data = torch.stack(tot_label)
        return input_data, label_data, rev_norm  
    
    
    
# ✅ 하이퍼파라미터 설정
hidden_size = 48 # LSTM의 hidden state 크기
dense_size = BATCH_SIZE * OUTPUT_SIZE
dropout_rate = 0.4
learning_rate = 0.01
epochs = 150
batch_size = BATCH_SIZE

# ✅ 1. PyTorch 모델 정의
class StockLSTM(nn.Module):
    def __init__(self):
        super(StockLSTM, self).__init__()
        self.lstm = nn.LSTM(INPUT_SIZE, hidden_size, batch_first=True)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, dense_size)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)
        self.output = nn.Linear(dense_size, OUTPUT_SIZE)  

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]  # 마지막 시점의 hidden state
        x = self.dropout1(last_output)
        x = self.relu(self.dense1(x))
        x = self.dropout2(x)
        out = self.output(x)
        return out

# ✅ 2. 데이터 준비 (Tensor로 변환)
train_loader = DataLoader(PriceData(train_df), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(PriceData(val_df), batch_size=batch_size)

# ✅ 3. 학습 준비
model = StockLSTM().to(DEVICE)
criterion = nn.L1Loss()  # MAE (mean absolute error)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ✅ 4. 학습 루프
best_val_loss = float('inf')
early_stop_counter = 0
patience = 10  # early stopping patience

pred_list = []
best_epoch = 0 
for epoch in range(epochs):
    pred_list_epoch = []
    model.train()
    running_loss = 0
    for xb, yb, zb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb, zb in val_loader:
            preds = model(xb)
            val_loss += criterion(preds, yb).item()
        ### min-max 정규화 
            local_min = zb[:,0,:]
            local_max = zb[:,1,:] 
            preds_real_val = preds * (local_max - local_min + 1e-2) + local_min 
            pred_list_epoch.append(preds_real_val.cpu().numpy())
            
    pred_list.append(pred_list_epoch) # 예측값 모음
    val_loss /= len(val_loader) # val loss 계산 
    print(f"[Epoch {epoch+1}] Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_epoch = epoch 
        best_val_loss = val_loss
        early_stop_counter = 0
        best_model_state = model.state_dict()
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("⏹ Early stopping triggered.")
            break

# ✅ 5. 최종 모델 저장 or 로드
model.load_state_dict(best_model_state)  # 가장 좋은 상태로 복원
# torch.save(model.state_dict(), "best_lstm_stock_model.pth")


         Date Price Open High  Low   Vol. Change %
0  2010-07-18   0.1    0  0.1  0.1  0.08K    0.00%
1  2010-07-19   0.1  0.1  0.1  0.1  0.57K    0.00%
2  2010-07-20   0.1  0.1  0.1  0.1  0.26K    0.00%
3  2010-07-21   0.1  0.1  0.1  0.1  0.58K    0.00%
4  2010-07-22   0.1  0.1  0.1  0.1  2.16K    0.00%


C:\Users\User\AppData\Local\Temp\ipykernel_24212\3313035048.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_val_norm_torch = torch.tensor(label_val_norm, dtype= torch.float32).to(DEVICE)
C:\Users\User\AppData\Local\Temp\ipykernel_24212\3313035048.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rev_norm.append(torch.tensor([input_val.min(), input_val.max()], dtype= torch.float32).to(DEVICE))
C:\Users\User\AppData\Local\Temp\ipykernel_24212\3313035048.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (con

[Epoch 1] Train Loss: 1238.4449, Val Loss: 0.1586
[Epoch 2] Train Loss: 1098.1831, Val Loss: 0.1167
[Epoch 3] Train Loss: 1020.2261, Val Loss: 0.1414
[Epoch 4] Train Loss: 1015.9958, Val Loss: 0.1467
[Epoch 5] Train Loss: 1000.2373, Val Loss: 0.1179
[Epoch 6] Train Loss: 998.5110, Val Loss: 0.1442
[Epoch 7] Train Loss: 1004.7308, Val Loss: 0.1127
[Epoch 8] Train Loss: 1003.4977, Val Loss: 0.1472
[Epoch 9] Train Loss: 1010.1607, Val Loss: 0.1312
[Epoch 10] Train Loss: 997.1656, Val Loss: 0.1237
[Epoch 11] Train Loss: 986.0037, Val Loss: 0.1517
[Epoch 12] Train Loss: 977.0782, Val Loss: 0.1307
[Epoch 13] Train Loss: 986.7628, Val Loss: 0.1409
[Epoch 14] Train Loss: 998.1437, Val Loss: 0.1339
[Epoch 15] Train Loss: 1601.2717, Val Loss: 0.3236
[Epoch 16] Train Loss: 1804.2002, Val Loss: 0.3326
[Epoch 17] Train Loss: 1716.7576, Val Loss: 0.3248
⏹ Early stopping triggered.


<All keys matched successfully>

In [3]:
save_dir = 'best_lstm_stock_model_light.pth'
torch.save(model.state_dict(), save_dir)

In [4]:
print(pred_list[best_epoch])
pd.DataFrame(np.vstack(pred_list[best_epoch]).reshape(-1, 4)).to_csv('pred.csv', index = False)

[array([[88229.69 , 87589.836, 88789.73 , 82754.85 ]], dtype=float32), array([[90464.21, 90046.24, 91059.49, 85502.33]], dtype=float32), array([[95312.5 , 96065.44, 96240.13, 93424.57]], dtype=float32), array([[87895.305, 87523.01 , 88874.625, 82869.484]], dtype=float32), array([[83849.98 , 82570.21 , 84256.266, 77137.33 ]], dtype=float32), array([[84546.79, 83333.55, 86974.56, 82103.69]], dtype=float32), array([[81875.29 , 85705.68 , 87113.55 , 79154.164]], dtype=float32), array([[86272.79, 88538.38, 90170.2 , 82818.64]], dtype=float32), array([[89521.97 , 93625.37 , 94819.484, 89128.38 ]], dtype=float32), array([[97019.19 , 96643.28 , 97950.875, 93460.52 ]], dtype=float32), array([[97242.84 , 96868.414, 98277.27 , 93769.23 ]], dtype=float32), array([[97670.664, 97339.02 , 98763.11 , 94438.17 ]], dtype=float32), array([[ 99014.07,  99247.7 , 100292.21,  96385.79]], dtype=float32), array([[97510.625, 97082.24 , 98337.87 , 93796.98 ]], dtype=float32), array([[95825.07, 95355.19, 96638.0

In [ ]:
# ✅ 5. 최종 모델 저장 or 로드
save_dir = 'best_lstm_stock_model_light.pth'
model.load_state_dict(torch.load(save_dir))  # 가장 좋은 상태로 복원
# torch.save(model.state_dict(), "best_lstm_stock_model.pth")
val_df = pd.read_csv('Bitcoin Historical Data.csv', encoding = 'utf-8-sig').fillna(0)
val_df = val_df[(pd.to_datetime(val_df['Date']) > pd.to_datetime('2023-12-31')) & (pd.to_datetime(val_df['Date']) < pd.to_datetime('2024-01-31'))]
val_df = val_df.sort_values(by = 'Date', ascending = True)
iter_cnt = (pd.to_datetime('2025-03-05') - pd.to_datetime('2024-01-31')).days

# Edit 함수 새로 정의 
def edit(input_data):
    input_data = input_data.sort_values(by = 'Date', ascending = True)
    for col in input_data.columns:
        if input_data[col].dtype == 'object':
            input_data[col] = input_data[col].astype(str).str.replace(',', '').str.replace('₩', '')
    input_data = input_data[['Price', 'Open', 'High', 'Low']].astype(np.float32)
# input 
    input_val_norm = (input_data - input_data.min()+ 1e-2) / (input_data.max() - input_data.min()+ 1e-2)
    input_val_norm_torch = torch.tensor(input_val_norm.values, dtype= torch.float32).unsqueeze(0).to(DEVICE)
# data for re-interpret the normaliaztion 
    rev_norm = torch.tensor([input_data.min(), input_data.max()], dtype= torch.float32).unsqueeze(0).to(DEVICE)
    input_data = torch.tensor(input_val_norm_torch, dtype= torch.float32).to(DEVICE)
    return input_data, rev_norm  

criterion = nn.L1Loss()  # MAE (mean absolute error)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ✅ 4. 학습 루프
best_val_loss = float('inf')
early_stop_counter = 0
patience = 10  # early stopping patience

pred_list = []
best_epoch = 0 
for epoch in range(epochs):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for idx in range(iter_cnt): 
            new_xb, new_zb = edit(val_df.iloc[idx:idx + SEQ_LEN, :].reset_index(drop = True))
            preds = model(new_xb)
            new_date = (pd.Timedelta(days = int(idx)) + pd.to_datetime('2024-01-31')).strftime("%Y-%m-%d")
            pred_df = pd.DataFrame(preds.cpu().numpy(), columns = ['Price', 'Open', 'High', 'Low'])
            pred_df['Date'] = new_date
            val_df = pd.concat([val_df, pred_df], axis = 0, ignore_index = True)
        ### epoch 별로 예측값 저장
            local_min = new_zb[:,0,:]
            local_max = new_zb[:,1,:]
            preds_real_val = preds * (local_max - local_min + 1e-2) + local_min 
            pred_list_epoch.append(preds_real_val.cpu().numpy())
            
# 예측값 모음
    pred_list.append(pred_list_epoch)
# val loss 계산 
    val_loss /= len(val_loader)

    print(f"[Epoch {epoch+1}] Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}")

    ## Early stopping
    if val_loss < best_val_loss:
        best_epoch = epoch 
        best_val_loss = val_loss
        early_stop_counter = 0
        best_model_state = model.state_dict()
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("⏹ Early stopping triggered.")
            break
        
print(pred_list[best_epoch])
pd.DataFrame(np.vstack(pred_list[best_epoch]).reshape(-1, 4)).to_csv('pred2.csv', index = False)

C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_dir))  # 가장 좋은 상태로 복원
C

[Epoch 1] Train Loss: 1716.7576, Val Loss: 0.0000


C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rev_norm = torch.tensor([input_data.min(), input_data.max()], dtype= torch.float32).unsqueeze(0).to(DEVICE)
C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_data = torch.tensor(input_val_norm_torch, dtype= torch.float32).to(DEVICE)
C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior

[Epoch 2] Train Loss: 1716.7576, Val Loss: 0.0000


C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rev_norm = torch.tensor([input_data.min(), input_data.max()], dtype= torch.float32).unsqueeze(0).to(DEVICE)
C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_data = torch.tensor(input_val_norm_torch, dtype= torch.float32).to(DEVICE)
C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior

...

[Epoch 9] Train Loss: 1716.7576, Val Loss: 0.0000


C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rev_norm = torch.tensor([input_data.min(), input_data.max()], dtype= torch.float32).unsqueeze(0).to(DEVICE)
C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_data = torch.tensor(input_val_norm_torch, dtype= torch.float32).to(DEVICE)
C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior

[Epoch 10] Train Loss: 1716.7576, Val Loss: 0.0000


C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rev_norm = torch.tensor([input_data.min(), input_data.max()], dtype= torch.float32).unsqueeze(0).to(DEVICE)
C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_data = torch.tensor(input_val_norm_torch, dtype= torch.float32).to(DEVICE)
C:\Users\User\AppData\Local\Temp\ipykernel_24212\3837443666.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior

[Epoch 11] Train Loss: 1716.7576, Val Loss: 0.0000
⏹ Early stopping triggered.
[array([[94838.195, 95071.08 , 94650.14 , 90353.   ]], dtype=float32), array([[94838.195, 95071.08 , 94650.14 , 90353.   ]], dtype=float32), array([[94838.195, 95756.984, 94871.266, 92316.03 ]], dtype=float32), array([[95512.734, 97219.12 , 96594.06 , 93132.71 ]], dtype=float32), array([[96941.53, 97219.12, 96889.64, 94286.83]], dtype=float32), array([[96941.53, 97219.12, 97773.53, 95740.8 ]], dtype=float32), array([[96941.53, 98854.55, 98824.93, 95740.8 ]], dtype=float32), array([[ 98639.195,  99891.836, 100226.47 ,  97108.15 ]], dtype=float32), array([[ 99710.37 , 101421.984, 101996.22 ,  99026.266]], dtype=float32), array([[101287.49, 101496.56, 102061.76,  99616.24]], dtype=float32), array([[101361.58, 101496.56, 102446.26,  99616.24]], dtype=float32), array([[101361.58, 101496.56, 102446.26,  99616.24]], dtype=float32), array([[101361.58, 102318.31, 102446.26,  99616.24]], dtype=float32), array([[102165